# Create the features from Topic Modeling
* using by_postal_code dataset

# Import

In [1]:
import pandas as pd
import json
import numpy as np
from collections import Counter
import pickle
import math 
from sklearn.model_selection import train_test_split

from scipy import sparse as sp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-t3dur0l5 because the default path (/home/emma/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
RANDOM_STATE = 24

In [ ]:
# 導入資料
train_df = pd.read_pickle('/home/adam/Steph_C/my_thesis/data/Train_by_postoal_code_without_review_pointwise_v3_3.pkl')
test_df = pd.read_pickle('/home/adam/Steph_C/my_thesis/data/Test_by_postoal_code_without_review_pointwise_v3_3.pkl')

In [ ]:
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)
all_df = all_df.reset_index(drop=True)

In [ ]:
print(train_df.shape , test_df.shape , all_df.shape)

# Review Features

In [ ]:
from transformers import BertTokenizer, BertModel

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

## Cal Cosine Similarity 

In [ ]:
train_df.columns

In [ ]:
# Bert Cosine Similarity 

def bert_cosine_similarity(df):
    
    for idx , row in df.iterrows():
        
        
    return df

In [ ]:
# create Affinity and Complementary
from gensim.matutils import cossim

def get_cos_sim(df):
    
    df['tok_cosine_sim'] =''
    df['tok_cosine_sim_w_attr'] =''
    df['tok_cosine_sim_tfidf'] =''
    
    for i in range(len(df)):
        df['tok_cosine_sim'][i] = cossim(df.LDA_res_tokens[i], df.LDA_loc_tokens[i])
        df['tok_cosine_sim_w_attr'][i] = cossim(df.LDA_res_tokens_with_att[i], df.LDA_loc_tokens_with_att[i])
        df['tok_cosine_sim_tfidf'][i] = cossim(df.LDA_res_tokens_with_att_tfidf[i], df.LDA_loc_tokens_with_att_tfidf[i])
    
    return df

In [ ]:
train_df = get_cos_sim(train_df)
test_df = get_cos_sim(test_df)

## Affinity & Complementary

In [ ]:
def get_aff_comp(df) : 
    
    df['tok_affinity'] = ''
    df['tok_complementary'] = ''
    df['tok_affinity_with_att'] = ''
    df['tok_complementary_with_att'] = ''
    df['tok_affinity_tfidf'] = ''
    df['tok_complementary_tfidf'] = ''
    
    for i in range(len(df)):
        affinity = 0
        complementary = 0
        affinity_with_att = 0
        complementary_with_att = 0
        affinity_tfidf = 0
        complementary_tfidf = 0
        
        for j in df.LDA_res_tokens[i]:
            for k in df.LDA_loc_tokens[i]: 
                if j[0] in k:
                    affinity += j[1]*k[1]
                    complementary += j[1]*(1-k[1])
        
        
        for j in df.LDA_res_tokens_with_att[i]:
            for k in df.LDA_loc_tokens_with_att[i]: 
                if j[0] in k:
                    affinity_with_att += j[1]*k[1]
                    complementary_with_att += j[1]*(1-k[1]) 
        
        for j in df.LDA_res_tokens_with_att_tfidf[i]:
            for k in df.LDA_loc_tokens_with_att_tfidf[i]: 
                if j[0] in k:
                    affinity_tfidf += j[1]*k[1]
                    complementary_tfidf += j[1]*(1-k[1]) 
        
        df['tok_affinity'][i] = affinity
        df['tok_complementary'][i] = complementary
        df['tok_affinity_with_att'][i] = affinity_with_att
        df['tok_complementary_with_att'][i] = complementary_with_att
        df['tok_affinity_tfidf'][i] = affinity_tfidf
        df['tok_complementary_tfidf'][i] = complementary_tfidf
        
    return df

        

In [ ]:
train_df = get_aff_comp(train_df)
test_df = get_aff_comp(test_df)

In [ ]:
for i in ['tok_affinity','tok_complementary','tok_affinity_with_att','tok_complementary_with_att','tok_cosine_sim','tok_affinity_tfidf','tok_complementary_tfidf']:
    train_df[i] = train_df[i].astype('float')
    test_df[i] = test_df[i].astype('float')

In [ ]:
train_df.to_pickle('/home/adam/Steph_C/my_thesis/data/Train_by_postoal_code_pointwise_v3_3_tfidf.pkl')
test_df.to_pickle('/home/adam/Steph_C/my_thesis/data/Test_by_postoal_code_pointwise_v3_3_tfidf.pkl')